### Analysis of pmdarima (Python's auto\_arima implementation) performance with COVID19 data from Brazil

In this Notebook it was implemented code to compare prediction results from Python's `auto_arima` implementation using COVID-19 data from Brazil against the two following articles:

- [Short-term forecasting COVID-19 cumulative confirmed cases: Perspectives for Brazil
](https://www.sciencedirect.com/science/article/pii/S0960077920302538)
- [Short-term forecasting of daily COVID-19 cases in Brazil by using the Holt’s mode](https://www.scielo.br/scielo.php?script=sci_arttext&pid=S0037-86822020000100643)

The Python's library is `pmdarima` package, version `1.8.0`

The data used was downloaded from [Ministry of Health of Brazil](https://covid.saude.gov.br) and saved as `covid.csv` file

In [1]:
import pmdarima as pmd
import numpy as np
import pandas as pd

#### Loading and checking data

In [2]:
filename = 'covid.csv'
full_df = pd.read_csv(filename, low_memory=False)

full_df

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125,0,0,0,0,NaN,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125,1,1,0,0,NaN,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125,1,0,0,0,NaN,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125,1,0,0,0,NaN,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125,2,1,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806975,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2020-08-13,33,3015268,131170,1986,1905,35,NaN,NaN,1.0
806976,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2020-08-14,33,3015268,133166,1996,1935,30,NaN,NaN,1.0
806977,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2020-08-15,33,3015268,135014,1848,1958,23,NaN,NaN,1.0
806978,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2020-08-16,34,3015268,136467,1453,1976,18,NaN,NaN,1.0


#### Helper functions

In [3]:
def filter_by_state(df, state, filter_zero=False):
    """
    Logic: Doesn't have "codmun" (county code) and match "estado" (state)

    Args:
        df (pandas.DataFrame): Global "full" dataframe, loaded from covid.csv file
        state (str): Brazilians abbreviated states (two uppercase chars, e.g. SP)
        filter_zero (bool): If true, removes cumulative cases where the values are zero

    Returns:
        Filtered dataframe by state and with "data" (portuguese word for date) column as index
    """
    filt_df = df[df['codmun'].isnull() & (df['estado'] == state)]
    if filter_zero:
        filt_df = filt_df[filt_df['casosAcumulado'] != 0]
    filt_df = filt_df.drop_duplicates('data')

    filt_df['data'] = pd.to_datetime(filt_df['data'], format='%Y-%m-%d')
    filt_df.set_index('data', inplace=True)
    return filt_df


def filter_brazil(df, filter_zero=False):
    """
    Logic: "regiao" (region) equals "Brasil"

    Args:
        df (pandas.DataFrame): Global "full" dataframe, loaded from covid.csv file
        state (str): Brazilians abbreviated states (two uppercase chars, e.g. SP)
        filter_zero (bool): If true, removes cumulative cases where the values are zero

    Returns:
        Filtered dataframe for Brazil with "data" (portuguese word for date) column as index
    """
    filt_df = df[df['regiao'] == 'Brasil']
    if filter_zero:
        filt_df = filt_df[filt_df['casosAcumulado'] != 0]
    filt_df = filt_df.drop_duplicates('data')

    filt_df['data'] = pd.to_datetime(filt_df['data'], format='%Y-%m-%d')
    filt_df.set_index('data', inplace=True)
    return filt_df


def predict(df, start_date, end_date, pred_days):
    """
    Predicts COVID-19 cumulative cases using auto_arima for the filtered dataframe

    The provided dataframe will be sliced into "start_date" and "end_date" as indexes
    The result slice will be divided into train and test data based on "pred_days":
        train data will be used to fit the model and test data will be used to
        compare and extract the smape metric against the prediction

    Args:
        df (pandas.DataFrame): A filtered DataFrame
        start_date (str): string with the following format: %Y-%m%-d (e.g. 2020-04-19)
        end_date (str): string with same format as start_date
        pred_days (int): number of days that will be predicted

    Returns:
        Dict with with following keys: prediction, smape and arima_order
    """
    cases_array = df.loc[start_date:end_date]['casosAcumulado'].values
    train, test = cases_array[:-pred_days], cases_array[-pred_days:]
    fit = pmd.auto_arima(train.astype('float32'))
    prediction = fit.predict(pred_days)
    res = pmd.metrics.smape(test, prediction)
    return {
        'prediction': prediction,
        'smape': res,
        'arima_order': fit.order
    }

### Short-term forecasting COVID-19 cumulative confirmed cases: Perspectives for Brazil
Article link: https://www.sciencedirect.com/science/article/pii/S0960077920302538

To compare the Python's auto_arima results with the article's results, it was implemented a routine to predict the number of cumulative cases for the same state, dates, number of predicted days as the article. After the predictions, it was calculated the smape metric between the predictions and the real data.

In the article, several predictions for COVID-19 cumulative cases were done. It was predicted based on available data in that moment for several states from Brazil: Amazonas (AM), Bahia (BA), Ceará (CE), Minas Gerais (MG), Paraná (PR), Rio de janeiro (RJ), Rio Grande do Norte (RN), Rio Grande do Sul (RS), Santa Catarina (SC) and São Paulo (SP).


The article also predicts based on different estimators: ARIMA, CUBIST, RF, RIDGE, Stacking, SVR. In this comparison we are only selecting the article's ARIMA's results 

In [4]:
# A table with "state", "start date" and "end date" data from the article where passed
# to a named tuple object called StateData in order to facilitate the analysis
from collections import namedtuple

StateData = namedtuple('StateData', ['state', 'start_date', 'end_date'])

states_data = [
    StateData(state='AM', start_date='2020-03-13', end_date='2020-04-19'),
    StateData(state='BA', start_date='2020-03-06', end_date='2020-04-19'),
    StateData(state='CE', start_date='2020-03-16', end_date='2020-04-19'),
    StateData(state='MG', start_date='2020-03-08', end_date='2020-04-19'),
    StateData(state='PR', start_date='2020-03-12', end_date='2020-04-18'),
    StateData(state='RJ', start_date='2020-03-05', end_date='2020-04-19'),
    StateData(state='RN', start_date='2020-03-12', end_date='2020-04-18'),
    StateData(state='RS', start_date='2020-03-10', end_date='2020-04-19'),
    StateData(state='SC', start_date='2020-03-12', end_date='2020-04-19'),
    StateData(state='SP', start_date='2020-02-25', end_date='2020-04-19')
]

In [5]:
print('Results for 1 day')
for state_data in states_data:
    state_df = filter_by_state(full_df, state_data.state)
    res = predict(state_df, state_data.start_date, state_data.end_date, pred_days=1)
    print('%s - ARIMA%s - sMAPE: %s' % (state_data.state, res['arima_order'], res['smape']))

Results for 1 day
AM - ARIMA(1, 2, 0) - sMAPE: 2.853771123088597
BA - ARIMA(0, 2, 1) - sMAPE: 7.024365409817703
CE - ARIMA(1, 2, 0) - sMAPE: 3.6361667360872687
MG - ARIMA(0, 2, 3) - sMAPE: 2.873270035554388
PR - ARIMA(0, 2, 1) - sMAPE: 4.088562098078861
RJ - ARIMA(2, 2, 2) - sMAPE: 1.7162745845322167
RN - ARIMA(0, 2, 1) - sMAPE: 4.347176624348275
RS - ARIMA(0, 2, 3) - sMAPE: 1.0041421940773971
SC - ARIMA(0, 2, 1) - sMAPE: 2.6997299797208862
SP - ARIMA(0, 2, 1) - sMAPE: 3.660998626710932


In [6]:
print('Results for 1 day')
for state_data in states_data:
    state_df = filter_by_state(full_df, state_data.state)
    res = predict(state_df, state_data.start_date, state_data.end_date, pred_days=1)
    print('%s - ARIMA%s - sMAPE: %s' % (state_data.state, res['arima_order'], res['smape']))

Results for 1 day
AM - ARIMA(1, 2, 0) - sMAPE: 2.853771123088597
BA - ARIMA(0, 2, 1) - sMAPE: 7.024365409817703
CE - ARIMA(1, 2, 0) - sMAPE: 3.6361667360872687
MG - ARIMA(0, 2, 3) - sMAPE: 2.873270035554388
PR - ARIMA(0, 2, 1) - sMAPE: 4.088562098078861
RJ - ARIMA(2, 2, 2) - sMAPE: 1.7162745845322167
RN - ARIMA(0, 2, 1) - sMAPE: 4.347176624348275
RS - ARIMA(0, 2, 3) - sMAPE: 1.0041421940773971
SC - ARIMA(0, 2, 1) - sMAPE: 2.6997299797208862
SP - ARIMA(0, 2, 1) - sMAPE: 3.660998626710932


In [7]:
print('Results for 3 days')
for state_data in states_data:
    state_df = filter_by_state(full_df, state_data.state)
    res = predict(state_df, state_data.start_date, state_data.end_date, pred_days=3)
    print('%s - ARIMA%s - sMAPE: %s' % (state_data.state, res['arima_order'], res['smape']))

Results for 3 days
AM - ARIMA(1, 2, 0) - sMAPE: 1.3183252603310427
BA - ARIMA(2, 2, 0) - sMAPE: 2.203271727286451
CE - ARIMA(2, 2, 0) - sMAPE: 6.487606123636027
MG - ARIMA(0, 2, 2) - sMAPE: 4.4317986104711125
PR - ARIMA(0, 2, 1) - sMAPE: 2.91911589673431
RJ - ARIMA(1, 2, 2) - sMAPE: 1.5306286737158497
RN - ARIMA(1, 2, 1) - sMAPE: 7.069237053244531
RS - ARIMA(0, 2, 3) - sMAPE: 0.5155951693431876
SC - ARIMA(0, 2, 1) - sMAPE: 1.1454918156355303
SP - ARIMA(2, 2, 1) - sMAPE: 10.139626416451057


In [8]:
print('Predictions for 6 days')
for state_data in states_data:
    state_df = filter_by_state(full_df, state_data.state)
    res = predict(state_df, state_data.start_date, state_data.end_date, pred_days=6)
    print('%s - ARIMA%s - sMAPE: %s' % (state_data.state, res['arima_order'], res['smape']))

Predictions for 6 days
AM - ARIMA(1, 2, 0) - sMAPE: 7.092452040482091
BA - ARIMA(2, 2, 2) - sMAPE: 11.56505517325585
CE - ARIMA(0, 2, 2) - sMAPE: 17.760496276764385
MG - ARIMA(2, 2, 0) - sMAPE: 6.453437760618954
PR - ARIMA(0, 2, 1) - sMAPE: 8.34343040734601
RJ - ARIMA(1, 2, 3) - sMAPE: 3.03632740534907
RN - ARIMA(1, 1, 0) - sMAPE: 19.62120992742239
RS - ARIMA(2, 2, 1) - sMAPE: 11.05747133723962
SC - ARIMA(0, 2, 1) - sMAPE: 3.4715926249892965
SP - ARIMA(1, 2, 1) - sMAPE: 19.753858814467925


### Results
The sMAPE was provided as Appendix A from the Ribeiro's article. The table below compares the sMAPE results from Python's auto_arima to the article's ARIMA's sMAPE

By empirically considering sMAPE differences of less than 1% with the same performance, we can conclude that 12 predictions was better, 8 was the same and 10 predictions had inferior performance.


| State  | N days of prediction  |  sMAPE auto_arima | sMAPE (RIBEIRO et al, 2020) |
| :----: | :-------------------: | :---------------: | :-------------------------: |
|        |           1           |        2.85       |            6.61             |
|  AM    |           3           |        1.32       |            6.55             |
|        |           6           |        7.09       |            6.97             |
|        |           1           |        7.02       |            1.56             |
|  BA    |           3           |        2.20       |            8.00             |
|        |           6           |       11.56       |           15.41             |
|        |           1           |        3.64       |            0.87             |
|  CE    |           3           |        6.49       |            3.01             |
|        |           6           |       17.76       |            9.34             |
|        |           1           |        2.87       |            3.63             |
|  MG    |           3           |        4.43       |            3.08             |
|        |           6           |        6.45       |            5.43             |
|        |           1           |        4.01       |            3.96             |
|  PR    |           3           |        2.92       |            6.21             |
|        |           6           |        8.34       |            8.20             |
|        |           1           |        1.72       |            3.17             |
|  RJ    |           3           |        1.53       |            3.18             |
|        |           6           |        3.04       |            3.67             |
|        |           1           |        4.35       |            1.61             |
|  RN    |           3           |        7.07       |            2.11             |
|        |           6           |       19.62       |            7.61             |
|        |           1           |        1.00       |            1.64             |
|  RS    |           3           |        0.52       |            3.22             |
|        |           6           |       11.05       |            4.31             |
|        |           1           |        2.70       |            2.43             |
|  SC    |           3           |        1.15       |            4.76             |
|        |           6           |        3.47       |            5.65             |
|        |           1           |        3.66       |            4.65             |
|  SP    |           3           |       10.14       |           14.56             |
|        |           6           |       19.75       |           27.74             |


### Short-term forecasting of daily COVID-19 cases in Brazil by using the Holt’s model

https://www.scielo.br/scielo.php?script=sci_arttext&pid=S0037-86822020000100643

In [9]:
states = ['SP', 'MG', 'RJ']
start_date='2020-01-01'
end_date='2020-05-03'

print('Predictions for 8 days')

# Results for Brazil
br_df = filter_brazil(full_df)
res = predict(br_df, start_date, end_date, pred_days=8)
print('BR - ARIMA%s - sMAPE: %s' % (res['arima_order'], res['smape']))


for state in states:
    state_df = filter_by_state(full_df, state, filter_zero=True)
    res = predict(state_df, start_date, end_date, pred_days=8)
    print('%s - ARIMA%s - sMAPE: %s' % (state, res['arima_order'], res['smape']))

Predictions for 8 days
BR - ARIMA(1, 2, 0) - sMAPE: 1.6804520500749773
SP - ARIMA(1, 2, 1) - sMAPE: 5.623936162071448
MG - ARIMA(0, 2, 3) - sMAPE: 3.6264291829236317
RJ - ARIMA(2, 2, 3) - sMAPE: 8.321438703701347


##### Calculating sMAPE by the values given in the article

In [10]:
brazil_real = [61888, 66501, 71886, 78162, 85380, 91589, 96559, 101147]
brazil_forecasted = [63598.77, 68898.82, 74198.86, 79498.91, 84798.95, 90099.00, 95399.05, 100699.09]
print('sMAPE for BR:', pmd.metrics.smape(brazil_real, brazil_forecasted))

SP_real = [20715, 21696, 24041, 26158, 28698, 30374, 31174, 31772]
SP_forecasted = [21288.91, 22573.96, 23859.01, 25144.06, 26429.11, 27714.16, 28999.21, 30284.26]
print('sMAPE for SP:', pmd.metrics.smape(SP_real, SP_forecasted))

MG_real = [1548, 1586, 1649, 1758, 1827, 1935, 2023, 2118]
MG_forecasted = [1537.80, 1600.34, 1662.89, 1725.44, 1787.99, 1850.54, 1913.09, 1975.64]
print('sMAPE for MG:', pmd.metrics.smape(MG_real, MG_forecasted))

RJ_real = [7111, 7944, 8504, 8869, 9453, 10166, 10546, 11139]
RJ_forecasted = [7169.33, 7570.99, 7972.66, 8374.33, 8775.99, 9177.66, 9579.32, 9980.99]
print('sMAPE for RJ:', pmd.metrics.smape(RJ_real, RJ_forecasted))

sMAPE for BR: 1.888280586913778
sMAPE for SP: 5.1030513515506275
sMAPE for MG: 2.928723180486233
sMAPE for RJ: 7.003955308533456
